In [ ]:
import torch
import torch.nn as nn
import pandas as pd

In [ ]:
# LOSS_FUNCTION = nn.BCEWithLogitsLoss()

In [ ]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
    
    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self, item):
        return {
            "x": torch.tensor(self.features[item, :], dtype=torch.float),
            "y": torch.tensor(self.targets[item, :], dtype=torch.float),
        }

In [ ]:
class Engine:
    def __init__(self, model, optimizer, device):
        self.model = model
        self.optimizer = optimizer
        self.device = device
    

    @staticmethod
    def loss_fn(outputs, targets)):
        return nn.BCEWithLogitsLoss()(outputs, targets)
    
    def train(self, data_loader):
        self.model.train()
        final_loss = 0
        for data in data_loader:
            self.optimizer.zero_grad()
            inputs = data["x"].to(self.device)
            targets = data["y"].to(self.device)
            outputs = self.model(inputs)
            loss = self.loss_fn(outputs, targets)
            loss.backward()
            self.optimizer.step()
            final_loss += loss.item()
        return final_loss / len(data_loader)

    def evaluate(self, data_loader):
        self.model.eval()
        final_loss = 0
        for data in data_loader:
            inputs = data["x"].to(self.device)
            targets = data["y"].to(self.device)
            outputs = self.model(inputs)
            loss = self.loss_fn(outputs, targets)
            final_loss += loss.item()
        return final_loss / len(data_loader)

    # def add_dummies(data, column):
    #     ohe = pd.get_dummies(data[column])
    #     ohe_columns = [f"{column}_{c}" for c in ohe.columns]
    #     ohe.columns = ohe_columns
    #     data = data.drop(column, axis=1)
    #     data = data.join(ohe)
    #     return data

    def process_data(data):
        ohe = pd.get_dummies(data["cp_time"])
        ohe_columns = [f"cp_time_{c}" for c in ohe.columns]
        ohe.columns = ohe_columns
        data = data.drop(['cp_time'], axis=1)
        data = data.join(ohe)

        ohe = pd.get_dummies(data["cp_dose"])
        ohe_columns = [f"cp_dose_{c}" for c in ohe.columns]
        ohe.columns = ohe_columns
        data = data.drop(['cp_dose'], axis=1)
        data = data.join(ohe)

        ohe = pd.get_dummies(data["cp_type"])
        ohe_columns = [f"cp_type_{c}" for c in ohe.columns]
        ohe.columns = ohe_columns
        data = data.drop(['cp_type'], axis=1)
        data = data.join(ohe)
        return data
       



In [ ]:
class Model(nn.Module):
    def __init__(self, nfeatures, ntargets, nlayers, hidden_size, dropout):
        super().__init__()
        layers = []
        for _ in range(nlayers):
            if len(layers) == 0:
                layers.append(nn.Linear(nfeatures, hidden_size))
                layers.append(nn.BatchNorm1d(hidden_size))
                layers.append(nn.Dropout(dropout))
                layers.append(nn.ReLU())
            else:
                layers.append(nn.Linear(hidden_size, hidden_size))
                layers.append(nn.BatchNorm1d(hidden_size))
                layers.append(nn.Dropout(dropout))
                layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_size, ntargets))
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.model(x)


